<a href="https://colab.research.google.com/github/Smruthi3/END2/blob/main/Session7-Assignment/Assignment-Part2/Session7_Assignment_part2_CoQA_seq2seq_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd 
#from torchtext.legacy.data import Example,Field, BucketIterator,Dataset
from torchtext.legacy.data import Field, BucketIterator
from torchtext.legacy import data

import spacy
import numpy as np

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


##Data Preview :

CoQA is a large-scale dataset for building Conversational Question Answering systems. The goal of the CoQA challenge is to measure the ability of machines to understand a text passage and answer a series of interconnected questions that appear in a conversation.

The the following links contains training and testing dataset :

Train : 'http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json'

Test : 'http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json'

As per the assignment instractions we have to spilt the data into 70% train and 30% test

To adhere this, combining both the links into one dataset and then split data into 70% train and 30% test using torch dataset

Following way data is prepared and fed to model:


1. The dataset contains story (conversation) , questions and answers. Note     that one story contains multiple questions and respective answers

2. #Model takes source and target as inputs:
  source here would be story and questions combined and given
  target here would be answers

  For example : If a story has 5 questions, then story is repeated 5 times with each question combined as source and respective answer to the question as the target









In [3]:
import pandas as pd
train=pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
test=pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json')

combine_data = pd.concat([train,test])
combine_data=combine_data.reset_index(drop=True)

In [4]:
train.shape, test.shape,combine_data.shape

((7199, 2), (500, 2), (7699, 2))

In [5]:
combine_data.data[0]

{'answers': [{'input_text': 'It was formally established in 1475',
   'span_end': 179,
   'span_start': 151,
   'span_text': 'Formally established in 1475',
   'turn_id': 1},
  {'input_text': 'research',
   'span_end': 494,
   'span_start': 454,
   'span_text': 'he Vatican Library is a research library',
   'turn_id': 2},
  {'input_text': 'history, and law',
   'span_end': 511,
   'span_start': 457,
   'span_text': 'Vatican Library is a research library for history, law',
   'turn_id': 3},
  {'input_text': 'philosophy, science and theology',
   'span_end': 545,
   'span_start': 457,
   'span_text': 'Vatican Library is a research library for history, law, philosophy, science and theology',
   'turn_id': 4},
  {'input_text': 'a  project',
   'span_end': 879,
   'span_start': 769,
   'span_text': 'March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts',
   'turn_id': 5},
  {'input_text': 'into periods',
   'span_end': 1127,
   'span_

In [6]:
import time
def data_prep(dt):
  st=time.time()
  story=[]
  question=[]
  answer=[]
  for i in range(0,dt.data.shape[0]):
    sty=dt.data[i]['story']
    for j in range(0,len(dt.data[i]['questions'])):
      story.append(sty)
      question.append(dt.data[i]['questions'][j]['input_text'])
      answer.append(dt.data[i]['answers'][j]['input_text'])
  print(time.time()-st)
  return(story,question,answer)


In [7]:
st_complete,question_complete,answer_complete=data_prep(combine_data)
coqa_dataset=pd.DataFrame(
    {'story': st_complete,
     'question': question_complete,
     'answer': answer_complete
    })

2.760836362838745


In [8]:
st_complete[0]

'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of t

In [9]:
question_complete[0]

'When was the Vat formally opened?'

In [10]:
answer_complete[0]

'It was formally established in 1475'

In [11]:
coqa_dataset.shape

(116630, 3)

In [12]:
#MAX_LEN = 100
#coqa_dataset['story'] = coqa_dataset['story'].apply(lambda x: ' '.join(x.split(' ')[:MAX_LEN]))

In [13]:
len(coqa_dataset['story'][0])

1385

In [14]:
coqa_dataset['sty_ques']=coqa_dataset['story']+" "+coqa_dataset['question']

In [15]:
coqa_dataset.head()

,story,question,answer,sty_ques
0,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items. \n\nScholars have traditionally divided the history of the library into five periods, Pre-Lateran, Lateran, Avignon, Pre-Vatican and Vatican. \n\nThe Pre-Lateran period, comprising the initial days of the library, dated from the earliest days of the Church. Only a handful of volumes survive from this period, though some are very significant.",When was the Vat formally opened?,It was formally established in 1475,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items. \n\nScholars have traditionally divided the history of the library into five periods, Pre-Lateran, Lateran, Avignon, Pre-Vatican and Vatican. \n\nThe Pre-Lateran period, comprising the initial days of the library, dated from the earliest days of the Church. Only a handful of volumes survive from this period, though some are very significant. When was the Vat formally opened?"
1,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items. \n\nScholar

In [16]:
coqa_dataset.sty_ques[0]

'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of t

####Obtaining only revelent fields from the data set

In [17]:
coqa_dataset1 = coqa_dataset[['sty_ques','answer']]


In [18]:
coqa_dataset1.shape

(116630, 2)

### Create our fields to process our data


In [19]:
question = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)
answer = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [20]:
fields = [('questions', question),('answers',answer)]

In [21]:
fields

[('questions', <torchtext.legacy.data.field.Field at 0x7f8197338650>),
 ('answers', <torchtext.legacy.data.field.Field at 0x7f81973030d0>)]

In [22]:
example = [data.Example.fromlist([coqa_dataset1.sty_ques[i],coqa_dataset1.answer[i]], fields) for i in range(coqa_dataset1.shape[0])]

In [23]:
questionDataset = data.Dataset(example, fields)

In [24]:
(train, test) = questionDataset.split(split_ratio=[0.70, 0.30], random_state=random.seed(SEED))

In [25]:
print(len(train))

81641


In [26]:
print(len(test))

34989


In [27]:
print(vars(train.examples[0]))

{'questions': ['the', 'weather', 'was', 'perfect', '.', 'we', 'were', 'just', 'out', 'for', 'a', 'picnic', '.', 'when', 'lunch', 'time', 'came', ',', 'we', 'decided', 'to', 'go', 'and', 'get', 'something', 'to', 'eat', ',', 'and', 'meet', 'back', 'on', 'the', 'grass', '.', 'susan', 'and', 'i', 'headed', 'for', 'a', 'hot', 'dog', 'stand', '.', 'we', 'watched', 'the', 'seller', 'put', 'together', 'the', 'hot', 'dog', '.', 'but', 'when', 'susan', 'took', 'out', 'her', 'money', ',', 'the', 'man', 'surprised', 'us', '.', '"', 'it', 'looks', 'a', 'little', 'overdone', ',', '"', 'he', 'said', ',', '"', 'so', 'you', 'do', "n't", 'have', 'to', 'pay', 'me', '.', '"', 'we', 'said', 'our', 'thanks', ',', 'joined', 'our', 'friends', ',', 'and', 'began', 'to', 'enjoy', 'our', 'food', '.', 'but', 'as', 'we', 'talked', 'and', 'ate', ',', 'i', 'noticed', 'a', 'man', 'sitting', 'alone', ',', 'in', 'dirty', 'clothes', '.', 'i', 'could', 'tell', 'that', 'he', 'had', "n't", 'had', 'a', 'bath', 'for', 'days

In [28]:
print(vars(train.examples[1]))

{'questions': ['hong', 'kong', '(', 'cnn', ')', '--', 'a', 'car', 'theft', 'in', 'china', 'that', 'spiraled', 'into', 'a', 'manhunt', 'and', 'a', 'baby', "'s", 'killing', 'has', 'left', 'the', 'country', 'grieving', 'and', 'sparked', 'outrage', 'on', 'social', 'media', '.', '\n\n', 'the', 'body', 'of', 'the', 'infant', ',', 'whose', 'name', 'was', 'haobo', ',', 'was', 'found', 'buried', 'in', 'the', 'snow', 'wednesday', '.', 'his', 'father', ',', 'xu', 'jialin', ',', 'said', 'he', 'and', 'his', 'wife', 'identified', 'their', 'two', '-', 'month', '-', 'old', 'son', ',', 'according', 'to', 'state', '-', 'run', 'news', 'agency', 'xinhua', '.', '\n\n', 'thousands', 'gathered', 'in', 'a', 'square', 'tuesday', 'night', 'in', 'changchun', ',', 'the', 'capital', 'of', 'jilin', 'province', ',', 'to', 'mourn', 'the', 'baby', ',', 'killed', 'by', '"', 'a', 'thief', 'who', 'found', 'the', 'missing', 'boy', 'sleeping', 'in', 'a', 'car', 'he', 'had', 'stolen', ',', '"', 'xinhua', 'reported', '.', '\

###Building Vocabulary

In [29]:
MAX_VOCAB_SIZE = 25_000

question.build_vocab(train,max_size = MAX_VOCAB_SIZE, 
                     min_freq = 2)
answer.build_vocab(train,max_size = MAX_VOCAB_SIZE,
                   min_freq = 2)


In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
device

device(type='cuda')

In [32]:
BATCH_SIZE = 64

train_iterator, test_iterator = BucketIterator.splits(
    (train, test), 
    batch_size = BATCH_SIZE, 
    device = device,sort=False)

###Defining The model arrchitecture

In [33]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [34]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]

        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [35]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [36]:
INPUT_DIM = len(question.vocab)
OUTPUT_DIM = len(answer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [37]:
print(INPUT_DIM)
print(OUTPUT_DIM)

25004
13002


In [38]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(25004, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(13002, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=13002, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

### Model Training and Evaluation

In [39]:
optimizer = optim.Adam(model.parameters())

In [40]:
TRG_PAD_IDX = answer.vocab.stoi[answer.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

#### Training loop

In [41]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.questions
        trg = batch.answers
              
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

#### Evaluation loop

In [42]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.questions
            trg = batch.answers

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [43]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Model training and Evaluation

In [44]:
N_EPOCHS = 10
CLIP = 1

best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    test_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {test_loss:.3f} |  Val. PPL: {math.exp(test_loss):7.3f}')

Epoch: 01 | Time: 12m 39s
	Train Loss: 5.457 | Train PPL: 234.375
	 Val. Loss: 5.202 |  Val. PPL: 181.608
Epoch: 02 | Time: 12m 38s
	Train Loss: 5.130 | Train PPL: 168.990
	 Val. Loss: 5.196 |  Val. PPL: 180.464
Epoch: 03 | Time: 12m 41s
	Train Loss: 4.997 | Train PPL: 147.912
	 Val. Loss: 5.194 |  Val. PPL: 180.139
Epoch: 04 | Time: 12m 46s
	Train Loss: 4.816 | Train PPL: 123.454
	 Val. Loss: 4.881 |  Val. PPL: 131.808
Epoch: 05 | Time: 12m 49s
	Train Loss: 4.483 | Train PPL:  88.493
	 Val. Loss: 4.732 |  Val. PPL: 113.493
Epoch: 06 | Time: 12m 44s
	Train Loss: 4.258 | Train PPL:  70.694
	 Val. Loss: 4.684 |  Val. PPL: 108.238
Epoch: 07 | Time: 12m 43s
	Train Loss: 4.077 | Train PPL:  58.965
	 Val. Loss: 4.644 |  Val. PPL: 103.951
Epoch: 08 | Time: 12m 46s
	Train Loss: 3.912 | Train PPL:  50.023
	 Val. Loss: 4.637 |  Val. PPL: 103.183
Epoch: 09 | Time: 12m 39s
	Train Loss: 3.754 | Train PPL:  42.674
	 Val. Loss: 4.637 |  Val. PPL: 103.266
Epoch: 10 | Time: 12m 46s
	Train Loss: 3.585 |